You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```

Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:

```
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
```

In the above diagram, the symbols indicate whether the path exits each square moving up (`^`), down (`v`), left (`<`), or right (`>`). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

***What is the fewest steps required to move from your current position to the location that should get the best signal?***

In [29]:
import numpy as np
import string
input_file  = open("data/2022-12-12-input.txt")
input_lines = input_file.readlines()
heights = np.array([list(x.strip()) for x in input_lines])
[start_x, start_y] = np.argwhere(heights == "S")[0]
[end_x, end_y] = np.argwhere(heights == "E")[0]
print(f'Start at ({start_x}, {start_y}), end at ({end_x}, {end_y})')

Start at (20, 0), end at (20, 148)


In [82]:
def PotentialNeighbors(x, y, min_x, max_x, min_y, max_y):
    return [[poss_x, poss_y] for [poss_x, poss_y]
            in [[x-1, y], [x+1, y], [x, y-1], [x, y+1]]
            if poss_x >= min_x and poss_x < max_x and poss_y >= min_y and poss_y < max_y
           ]
min_x = 0
max_x = np.shape(heights)[0]
min_y = 0
max_y = np.shape(heights)[1]

height_map = {v:k for k,v in enumerate(string.ascii_lowercase)}
height_map.update({'S': 0, 'E': 25})
numeric_heights = np.vectorize(height_map.get)(heights)
distances = np.full(np.shape(heights), 1000)

active_locations = [[start_x, start_y]]
distances[start_x, start_y] = 0

while len(active_locations) > 0:
    [cur_x, cur_y] = active_locations.pop(0)
    for [next_x, next_y] in PotentialNeighbors(cur_x, cur_y, min_x, max_x, min_y, max_y):
        if (numeric_heights[next_x, next_y] - numeric_heights[cur_x, cur_y] < 2
            and distances[next_x, next_y] == 1000):
            distances[next_x, next_y] = distances[cur_x, cur_y] + 1
            active_locations.append([next_x, next_y])

print(f'The number of steps to get to the best location is {distances[end_x, end_y]}')

The number of steps to get to the best location is 490


As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:

```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```

Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:

```
...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^
```

This path reaches the goal in only 29 steps, the fewest possible.

***What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?***

In [105]:
def ShortestPath(from_x, from_y, to_x, to_y, numeric_heights):
    min_x = 0
    min_y = 0
    [max_x, max_y] = np.shape(numeric_heights)
    active_locations = [[from_x, from_y]]
    distances = np.full(np.shape(numeric_heights), 1000)
    distances[from_x, from_y] = 0
    while len(active_locations) > 0:
        [cur_x, cur_y] = active_locations.pop(0)
        for [next_x, next_y] in PotentialNeighbors(cur_x, cur_y, min_x, max_x, min_y, max_y):
            if (numeric_heights[next_x, next_y] - numeric_heights[cur_x, cur_y] < 2
                and distances[next_x, next_y] == 1000):
                if (next_x, next_y) == (to_x, to_y):
                    return distances[cur_x, cur_y] + 1
                distances[next_x, next_y] = distances[cur_x, cur_y] + 1
                active_locations.append([next_x, next_y])
    return 10000
    
shortest_paths = [ShortestPath(low_x, low_y, end_x, end_y, numeric_heights) for [low_x, low_y] in np.argwhere(numeric_heights == 0)]
print(f'The fewest steps required to move from a square with elevation a to the best location is {min(shortest_paths)}')

The fewest steps required to move from a square with elevation a to the best location is 488
